# Client 

### by 
### Nishanth.S and Shruthi.R
#### CB.EN.U4AIE22149 , CB.EN.U4AIE22154
##### Batch-B

In [4]:
import tkinter as tk
from tkinter import filedialog
import socket
import hashlib
import os
import pickle

def encrypt_byte_string(data, shift):
    encrypted_data = bytearray()
    for byte in data:
        encrypted_byte = (byte + shift) % 256  # Apply right shift by 'shift' amount
        encrypted_data.append(encrypted_byte)
    return bytes(encrypted_data)

def decrypt_byte_string(encrypted_data, shift):
    decrypted_data = bytearray()
    for byte in encrypted_data:
        decrypted_byte = (byte - shift) % 256  # Apply reverse right shift by 'shift' amount
        decrypted_data.append(decrypted_byte)
    return bytes(decrypted_data)

def myhash(data):
    out = hashlib.sha256(data.encode()).hexdigest()
    return out

def generate_merkle_tree(data_blocks):
    if len(data_blocks) == 0:
        return None
    
    if len(data_blocks) % 2 != 0:
        data_blocks.append(data_blocks[-1])
    
    hashed_blocks = [myhash(data) for data in data_blocks]
    length_hashed_blocks = len(hashed_blocks)
    
    while length_hashed_blocks > 1:
        if length_hashed_blocks % 2 != 0:
            hashed_blocks.append(hashed_blocks[-1])
        
        new_hashes = [myhash(hashed_blocks[i] + hashed_blocks[i + 1]) for i in range(0, length_hashed_blocks, 2)]
        hashed_blocks = new_hashes
        length_hashed_blocks = len(hashed_blocks)
    
    return hashed_blocks[0]

def uploadfile_window():
    def client_pgm(file_path):
        remote_host = "LAPTOP-SN01"  
        port = 5002
        client_socket = socket.socket()
        client_socket.connect((remote_host, port))
        opt="1"
        file_name = os.path.basename(file_path)  # Get the file name from the path
        directory= r"E:/client/binary files/"
        file_name2=""
        file_name2=file_name2.join(file_name.split(".")[:-1])
        file_name2=directory+file_name2+".bin"
        print(file_name2)
        filename_length = str(len(file_name))
        while len(filename_length)<3:
            filename_length="0"+filename_length
        client_socket.send(opt.encode())
        client_socket.send(filename_length.encode())
        # Send the file name to the server
        client_socket.send(file_name.encode())

        chunks=[]
        with open(file_path, 'rb') as file:
            data = file.read(128)
            while data:
                encrypted_data = encrypt_byte_string(data, 5)  # Encrypt the byte string with right shift 5
                client_socket.send(encrypted_data)
                chunks.append(str(data))
                data = file.read(128)

        #print(chunks)        
        merkle_root = generate_merkle_tree(chunks)
        
        file=open(file_name2, 'wb+')
        pickle.dump(merkle_root,file)
        file.close()
        print(f"File '{file_name}' sent to the server")

        client_socket.close()
        
    def browse_file():
        file_path = filedialog.askopenfilename()
        selected_file.set(file_path)

    def upload():
        file_path = selected_file.get()
        if file_path:
            client_pgm(file_path)
        else:
            print("No file selected")

    uploadfile_root = tk.Toplevel()
    uploadfile_root.title("Upload File")
    uploadfile_root.geometry("500x300")
    uploadfile_root.configure(bg="#FFFFFF")

    frame1 = tk.Frame(uploadfile_root, background="#FFFFFF")

    lbl1 = tk.Label(uploadfile_root, text="File upload", font=('Vladimir Script', 25, 'bold'), bg="#FFFFFF", fg='#00FFFF')
    btn1 = tk.Button(frame1, text="Select File", command=browse_file, font=('Calibri', 10), width=25)
    btn2 = tk.Button(frame1, text="Upload File", command=upload, font=('Calibri', 10), width=25)
    selected_file = tk.StringVar()  # Variable to store the selected file path
    lbl_selected_file = tk.Label(frame1, textvariable=selected_file, bg="#FFFFFF")

    lbl1.pack()
    btn1.pack(pady=20)
    btn2.pack()
    lbl_selected_file.pack()  
    frame1.pack()

    frame2 = tk.Frame(uploadfile_root, background="#AFFFFF")
    lbl_version = tk.Label(uploadfile_root, text="Version : 3.0.01", font=('Calibri', 15), bg="#FFFFFF")
    lbl_version.pack()
    frame2.pack()
    
def downloadfile_window():
    def client_download_pgm(selected_file):
        remote_host = "LAPTOP-SN01"
        port = 5002
        client_socket = socket.socket()
        client_socket.connect((remote_host, port))
        opt = "3"
        client_socket.send(opt.encode())
        client_socket.send(selected_file.encode())
        
        dirmerk=r"E:/client/binary files/"
        merkelfile_name=""
        merkelfile_name=merkelfile_name.join(selected_file.split(".")[:-1])
        file_path=dirmerk+merkelfile_name+".bin"
        
        file1=open(file_path, 'rb')
        d=pickle.load(file1)
        #print(d)
        file1.close()
        
        directory = filedialog.askdirectory()
        if directory:
            print("Selected directory:", directory)
            
        chunks=[]
        with open(directory+r"/"+selected_file, 'wb') as file:
            while True:
                encrypted_data = client_socket.recv(128)
                decrypted_data = decrypt_byte_string(encrypted_data, 5)  # Decrypt the byte string with reverse right shift 5
                if not decrypted_data:
                    break
                chunks.append(str(decrypted_data))
                file.write(decrypted_data)
            
        merkle_root = generate_merkle_tree(chunks)
        
        if merkle_root == d:
            print(f"File '{selected_file}' downloaded from the server")
            #os.remove(file_path)
        else:
            #os.remove(directory+r"/"+selected_file)
            print(f"File '{selected_file}' is tampered! removed the file from the folder.")

        client_socket.close()

    def select_file():
        global file_listbox
        selected_file = file_listbox.get(tk.ACTIVE)
        if selected_file:
            # Call the client_download_pgm function with the selected file name
            client_download_pgm(selected_file)
        else:
            print("No file selected")

    def display_files(files):
        global file_listbox
        downloadfile_root = tk.Toplevel()
        downloadfile_root.title("Download File")
        downloadfile_root.geometry("500x300")
        downloadfile_root.configure(bg="#FFFFFF")

        frame1 = tk.Frame(downloadfile_root, background="#FFFFFF")

        lbl1 = tk.Label(downloadfile_root, text="Download file", font=('Vladimir Script', 25, 'bold'), bg="#FFFFFF", fg='#00FFFF')

        file_listbox = tk.Listbox(frame1)
        file_listbox.pack(fill=tk.BOTH, expand=True)

        for file in files:
            file_listbox.insert(tk.END, file)

        select_button = tk.Button(frame1, text="Download Selected File", command=select_file, font=('Calibri', 10), width=25)
        lbl1.pack()
        file_listbox.pack(pady=20)
        select_button.pack()

        frame1.pack()

        frame2 = tk.Frame(downloadfile_root, background="#AFFFFF")
        lbl_version = tk.Label(downloadfile_root, text="Version : 3.0.01", font=('Calibri', 15), bg="#FFFFFF")
        lbl_version.pack()
        frame2.pack()

    def client_request_files():
        remote_host = "LAPTOP-SN01"
        port = 5002
        client_socket = socket.socket()
        client_socket.connect((remote_host, port))
        opt = "2"
        client_socket.send(opt.encode())
        files = client_socket.recv(1024).decode().split(",")

        display_files(files)

    client_request_files()

def main_window():
    main_root = tk.Tk()
    main_root.title("Client")
    main_root.geometry("500x300")
    main_root.configure(bg="#FFFFFF")
    frame1 = tk.Frame(main_root, background="#FFFFFF")

    lbl1 = tk.Label(main_root, text="Client", font=('Vladimir Script', 25, 'bold'), bg="#FFFFFF", fg='#00FFFF')
    btn1 = tk.Button(frame1, text="Upload Files", command=uploadfile_window, font=('Calibri', 10), width=25)
    btn2 = tk.Button(frame1, text="Download Files", command=downloadfile_window, font=('Calibri', 10), width=25)
    lbl1.pack()
    btn1.pack(pady=30)
    btn2.pack(pady=20)
    
    frame1.pack()
    frame2 = tk.Frame(main_root, background="#AFFFFF")
    lbl1 = tk.Label(frame2, text="Version : 3.0.01", font=('Calibri', 15), bg="#FFFFFF")
    lbl1.pack()
    frame2.pack()
    main_root.mainloop()

if __name__ == '__main__':
    main_window()
